### Reconstruction of one layer's TT and focus from TT sensing with two NGS

In [4]:
from apposto.utils.modal_decomposer import ModalDecomposer
from apposto.types.wavefront import Wavefront
from apposto.types.mask import CircularMask
from apposto.utils.zernike_generator import ZernikeGenerator
import numpy as np

In [5]:
mp_diameter = 1024
fp1_center = (512, 512 - 50)
fp1_radius = 100
fp2_center = (512, 512 + 50)
fp2_radius = fp1_radius
fp1_mask = CircularMask((mp_diameter, mp_diameter), fp1_radius, fp1_center)
fp2_mask = CircularMask((mp_diameter, mp_diameter), fp2_radius, fp2_center)

In [8]:
zg = ZernikeGenerator(mp_diameter)
measured_modes = 3 # Measurement of tip and tilt with NGS
m_modes = np.array([2, 3, 4]) # Correction of tip, tilt and focus
md = ModalDecomposer(n_zernike_modes=measured_modes)
im1 = np.zeros((measured_modes, m_modes.size))

for i in range(m_modes.size):
    wf = Wavefront(zg.getZernike(m_modes[i]))
    a1 = md.measureZernikeCoefficientsFromWavefront(wf, fp1_mask).toNumpyArray()
    im1[:, i] = a1
    
im2 = np.zeros((measured_modes, m_modes.size))
for i in range(m_modes.size):
    wf = Wavefront(zg.getZernike(m_modes[i]))
    a2 = md.measureZernikeCoefficientsFromWavefront(wf, fp2_mask).toNumpyArray()
    im2[:, i] = a2

print('Interaction matrix (NGS1): \n {} \n'.format(im1))
print('Reconstructor (NGS1): \n {} \n'.format(np.linalg.pinv(im1)))
print('Interaction matrix (NGS2): \n {} \n'.format(im2))
print('Reconstructor (NGS2): \n {} \n'.format(np.linalg.pinv(im2)))
print('r_FP/r_MP: {}'.format(fp1_radius / (mp_diameter / 2)))
print('(r_FP/r_MP)**2: {}'.format(fp1_radius**2 / (mp_diameter / 2)**2))

Interaction matrix (NGS1): 
 [[ 1.95312500e-01 -3.25260652e-19 -6.60724948e-02]
 [-1.72930247e-17  1.95312500e-01  3.46944695e-18]
 [-1.30646362e-17 -1.45716772e-16  3.81469559e-02]] 

Reconstructor (NGS1): 
 [[5.12000000e+00 9.39224702e-15 8.86810404e+00]
 [4.53326265e-16 5.12000000e+00 1.36426768e-16]
 [1.74660224e-15 2.79669749e-14 2.62144115e+01]] 

Interaction matrix (NGS2): 
 [[ 1.95312500e-01 -1.00288701e-18  6.60724948e-02]
 [-1.76182853e-17  1.95312500e-01 -1.99493200e-17]
 [-1.32814766e-17 -1.38777878e-16  3.81469559e-02]] 

Reconstructor (NGS2): 
 [[ 5.12000000e+00  1.52910645e-14 -8.86810404e+00]
 [ 4.61852778e-16  5.12000000e+00  4.41104674e-15]
 [ 1.40932728e-15 -4.59977150e-14  2.62144115e+01]] 

r_FP/r_MP: 0.1953125
(r_FP/r_MP)**2: 0.03814697265625


In [9]:
im_total = np.vstack((im1, im2))
reconstructor = np.linalg.pinv(im_total)
print('Interaction matrix: \n {} \n'.format(im_total))
print('Reconstructor: \n {} \n'.format(reconstructor))

Interaction matrix: 
 [[ 1.95312500e-01 -3.25260652e-19 -6.60724948e-02]
 [-1.72930247e-17  1.95312500e-01  3.46944695e-18]
 [-1.30646362e-17 -1.45716772e-16  3.81469559e-02]
 [ 1.95312500e-01 -1.00288701e-18  6.60724948e-02]
 [-1.76182853e-17  1.95312500e-01 -1.99493200e-17]
 [-1.32814766e-17 -1.38777878e-16  3.81469559e-02]] 

Reconstructor: 
 [[ 2.56000000e+00 -2.26663133e-16 -1.71240799e-16  2.56000000e+00
  -2.30926389e-16 -1.74082970e-16]
 [-1.88980446e-15  2.56000000e+00 -6.84201632e-16  2.34739398e-15
   2.56000000e+00 -5.93252161e-16]
 [-5.67558534e+00  3.84414645e-15  3.27679928e+00  5.67558534e+00
   1.83248925e-15  3.27679928e+00]] 

